##### Goal
Goal of this notebook is to narrow down the features. The results will then be used to trim down what columns the build script keeps.

What we can drop:

sup_num
sup_action
sup_cd
sup_desc

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Load in 2019 data from cleaned h5 file
bexar_df = pd.read_hdf('../data/processed/bexar_merged_df.h5')

In [3]:
print(bexar_df.columns)

Index(['prop_id', 'prop_type_cd', 'prop_val_yr', 'sup_num', 'sup_action',
       'sup_cd', 'sup_desc', 'geo_id', 'py_owner_id', 'py_owner_name',
       'partial_owner', 'udi_group', 'py_addr_line1', 'py_addr_line2',
       'py_addr_line3', 'py_addr_city', 'py_addr_state', 'py_addr_country',
       'py_addr_zip', 'py_addr_zip_cass', 'py_addr_zip_rt',
       'py_confidential_flag', 'py_address_suppress_flag',
       'py_addr_ml_deliverable', 'situs_street_prefx', 'situs_street',
       'situs_street_suffix', 'situs_city', 'situs_zip', 'hood_cd',
       'appraised_val', 'deed_dt', 'land_acres', 'entity_agent_id',
       'entity_agent_name', 'entity_agent_addr_line1',
       'entity_agent_addr_line2', 'entity_agent_addr_line3',
       'entity_agent_city', 'entity_agent_state', 'entity_agent_country',
       'ca_agent_id', 'ca_agent_name', 'ca_agent_addr_line1',
       'ca_agent_addr_line2', 'ca_agent_addr_line3', 'ca_agent_city',
       'ca_agent_state', 'ca_agent_country', 'ca_agent_zip',

#### Categorical Variables

##### Supplement info
Can we drop the supplement information?

In [4]:
# Yes
print(bexar_df.sup_cd.value_counts()[:10])
print(bexar_df.sup_action.value_counts())

FREEZE    18279
IA        13171
ARB       10711
A-EX       5410
D-EX       1057
CHG         613
OWN         544
CHAIR       369
FID         207
LIT         123
Name: sup_cd, dtype: int64
M    51014
A     4980
Name: sup_action, dtype: int64


In [5]:
bexar_df.sup_desc.value_counts()[:5]

Supplemented for 2019 to Refreeze Accounts after Tax Rates set                                     18131
ARB Decision                                                                                        7412
CHAIRMAN GRANTED 2019 LATE PROTEST. OPENED PROTEST & CH INQ SCANNED//SM                              165
Freeze Correction - Automatically added to this supplement because of prior year freeze change.      145
NEED A SUPPLEMENT CODE                                                                               139
Name: sup_desc, dtype: int64

In [6]:
bexar_df.drop(columns=['sup_num', 'sup_action','sup_cd', 'sup_desc'],inplace=True)

##### UDI Group

In [7]:
# Unclear what this is
bexar_df.udi_group.value_counts()

0          693140
182698         35
186585         22
160017         10
160016         10
            ...  
1234278         2
1144030         2
1224176         1
1015749         1
1136194         1
Name: udi_group, Length: 1102, dtype: int64

In [8]:
bexar_df[bexar_df.udi_group==160016][:5]

,prop_id,prop_type_cd,prop_val_yr,geo_id,py_owner_id,py_owner_name,partial_owner,udi_group,py_addr_line1,py_addr_line2,...,Officer/Director Name,Officer/Director Title,Officer/Director State,deed_charter_diff,just_established_owner,foreign_based_owner,yoy_diff_2019,yoy_diff_2018,yoy_diff_2017,yoy_diff_2016
500612,985472,R,2019,04009-000-0046,115923,WORD ANN SCHUPBACH,T,160016,NaN,708 TERRELL RD,...,NaN,NaN,NaN,NaT,0,0,326.0,0.0,2249.0,104.0
500613,985473,R,2019,04009-000-0046,115924,CANTY LYNN SCHUPBACH,T,160016,NaN,806 WILTSHIRE AVE,...,NaN,NaN,NaN,NaT,0,0,326.0,0.0,2249.0,104.0
500614,985474,R,2019,04009-000-0046,115926,SCHUPBACH ROBERT JAMES,T,160016,NaN,3 OSBORN HL,...,NaN,NaN,NaN,NaT,0,0,326.0,0.0,2249.0,104.0
500615,985475,R,2019,04009-000-0046,115920,SCHUPBACH HELEN JAMES,T,160016,NaN,223 LAMONT AVE,...,NaN,NaN,NaN,NaT,0,0,4570.0,0.0,31552.0,1459.0
500616,985476,R,2019,04009-000-0046,3045197,HEBDON ESPERANZA LLC,T,160016,NaN,8102 NUFY RDG,...,ELIZABETH ANNE WEST GST TRUST U T A DTD 1,MEMBER,TX,13 days,1,0,1958.0,0.0,13516.0,625.0


##### Owner Address

Will make another binary column for out of state owners

In [9]:
bexar_df[['py_addr_line1', 'py_addr_line2','py_addr_line3', 'py_addr_city', 'py_addr_state', 'py_addr_country']].head()

,py_addr_line1,py_addr_line2,py_addr_line3,py_addr_city,py_addr_state,py_addr_country
0,NaN,119 E CRAIG PL,NaN,SAN ANTONIO,TX,US
1,NaN,216 1/2 TERRY CT,NaN,SAN ANTONIO,TX,US
2,NaN,7 W KITTY HAWK ST,NaN,RICHMOND,TX,US
3,NaN,7 W KITTY HAWK ST,NaN,RICHMOND,TX,US
4,NaN,PO BOX 839966,NaN,SAN ANTONIO,TX,US


In [10]:
bexar_df.py_addr_state.value_counts()

TX           660613
CA             9279
CO             1702
FL             1689
IL             1216
              ...  
25730             1
COAH MEX          1
AS                1
MEXICO DF         1
JAL               1
Name: py_addr_state, Length: 64, dtype: int64

In [11]:
# corp_prop_merged['foreign_based_owner'] = np.where(corp_prop_merged['py_addr_country'] != 'US',1,0)
bexar_df['out_of_state_owner'] = np.where(bexar_df['py_addr_state'] != 'TX',1,0)

In [12]:
bexar_df['out_of_state_owner'].value_counts()

0    660613
1     34897
Name: out_of_state_owner, dtype: int64

##### Confidential Flag

In [13]:
bexar_df['py_confidential_flag'].value_counts()

F    594049
T      2424
Name: py_confidential_flag, dtype: int64

In [14]:
# Convert from T/F to 1/0
bexar_df['py_confidential_flag'] = np.where(bexar_df['py_confidential_flag'] == 'T',1,0)

In [15]:
bexar_df['py_confidential_flag'].value_counts()

0    693086
1      2424
Name: py_confidential_flag, dtype: int64

##### Property Year Owner Address Suppression Flag 

In [16]:
# Do the same for this
bexar_df['py_address_suppress_flag'] = np.where(bexar_df['py_address_suppress_flag']=='T',1,0)

In [17]:
bexar_df['py_address_suppress_flag'].value_counts()

0    691653
1      3857
Name: py_address_suppress_flag, dtype: int64

##### py_addr_ml_deliverable
("Property Year Owner Address
Y = deliverable address
N = undeliverable address"

In [18]:
bexar_df.py_addr_ml_deliverable.value_counts()

Y    595002
N      1809
Name: py_addr_ml_deliverable, dtype: int64

In [19]:
bexar_df['py_addr_ml_deliverable'] = np.where(bexar_df['py_addr_ml_deliverable']=='Y',1,0)
print(bexar_df['py_addr_ml_deliverable'].value_counts())

1    595002
0    100508
Name: py_addr_ml_deliverable, dtype: int64


##### Property Address

In [20]:
bexar_df[['situs_street_prefx', 'situs_street', 'situs_street_suffix', 'situs_city', 'situs_zip']].head()

,situs_street_prefx,situs_street,situs_street_suffix,situs_city,situs_zip
0,NaN,VALERO ST,NaN,SAN ANTONIO,78212
1,NaN,TERRY CT,NaN,SAN ANTONIO,78212
2,NaN,TERRY CT,NaN,SAN ANTONIO,78212
3,NaN,TERRY CT,NaN,SAN ANTONIO,78212
4,NaN,RIVER WAY,NaN,SAN ANTONIO,78212


In [23]:
print(bexar_df.shape)
print(bexar_df.situs_city.value_counts()[:5])
# ~82% of the properties are in the city of San Antonio

(695510, 81)
SAN ANTONIO       568247
CONVERSE           16803
HELOTES            11316
BOERNE              7350
UNIVERSAL CITY      6471
Name: situs_city, dtype: int64


In [22]:
# Too wide for get dummies?
# Maybe a binary based on San Antonio or not?
bexar_df.situs_city.nunique()

163

In [32]:
print(bexar_df.situs_zip.unique())

['78212' '78215' nan ... '78217-1223' '78266-2813' '78229-6108']


##### Neighborhood code (hood_cd)

In [24]:
bexar_df.hood_cd.nunique()

2768

In [25]:
# Still unsure what to do with these
bexar_df.hood_cd.value_counts()[:10]

24090    4359
57076    3743
57019    3653
57018    3384
57210    2628
95312    2506
95308    2501
57014    2498
57082    2486
28290    2466
Name: hood_cd, dtype: int64

##### Entity Agent
Making it binary if one exists or not

In [35]:
bexar_df.entity_agent_id.value_counts()

0          681187
2837462      1011
60100         737
2844946       590
60101         522
            ...  
3172018         1
3060073         1
3118201         1
3171207         1
40078           1
Name: entity_agent_id, Length: 904, dtype: int64

In [37]:
print(len(bexar_df.entity_agent_id))
print(len(bexar_df.entity_agent_id.dropna()))

695510
695510


In [38]:
bexar_df['entity_agent_binary'] = np.where(bexar_df.entity_agent_id==0,0,1)

In [39]:
bexar_df.entity_agent_binary.value_counts()

0    681187
1     14323
Name: entity_agent_binary, dtype: int64

In [40]:
bexar_df.drop(columns=['entity_agent_id','entity_agent_name', 
                       'entity_agent_addr_line1','entity_agent_addr_line2', 
                       'entity_agent_addr_line3','entity_agent_city', 
                       'entity_agent_state', 'entity_agent_country'],inplace=True)

In [41]:
bexar_df.shape

(695510, 74)

##### Chief Appraiser

In [42]:
bexar_df[['ca_agent_id', 'ca_agent_name', 'ca_agent_addr_line1',
       'ca_agent_addr_line2', 'ca_agent_addr_line3', 'ca_agent_city',
       'ca_agent_state', 'ca_agent_country', 'ca_agent_zip']].head()

,ca_agent_id,ca_agent_name,ca_agent_addr_line1,ca_agent_addr_line2,ca_agent_addr_line3,ca_agent_city,ca_agent_state,ca_agent_country,ca_agent_zip
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
bexar_df.ca_agent_id.value_counts()

0          626311
60585       12326
60001        6744
40773        5925
60075        2424
            ...  
3168658         1
3173093         1
40781           1
2663564         1
3169700         1
Name: ca_agent_id, Length: 1181, dtype: int64

In [46]:
bexar_df[bexar_df.ca_agent_id==60585][['ca_agent_id', 'ca_agent_name', 'ca_agent_addr_line1',
       'ca_agent_addr_line2', 'ca_agent_addr_line3', 'ca_agent_city',
       'ca_agent_state', 'ca_agent_country', 'ca_agent_zip']][:5]

,ca_agent_id,ca_agent_name,ca_agent_addr_line1,ca_agent_addr_line2,ca_agent_addr_line3,ca_agent_city,ca_agent_state,ca_agent_country,ca_agent_zip
7,60585,ARTHUR P. VELTMAN & ASSOCIATES,NaN,1017 N MAIN AVE STE 201,NaN,SAN ANTONIO,TX,US,78212.0
8,60585,ARTHUR P. VELTMAN & ASSOCIATES,NaN,1017 N MAIN AVE STE 201,NaN,SAN ANTONIO,TX,US,78212.0
9,60585,ARTHUR P. VELTMAN & ASSOCIATES,NaN,1017 N MAIN AVE STE 201,NaN,SAN ANTONIO,TX,US,78212.0
14,60585,ARTHUR P. VELTMAN & ASSOCIATES,NaN,1017 N MAIN AVE STE 201,NaN,SAN ANTONIO,TX,US,78212.0
49,60585,ARTHUR P. VELTMAN & ASSOCIATES,NaN,1017 N MAIN AVE STE 201,NaN,SAN ANTONIO,TX,US,78212.0


In [47]:
bexar_df.ca_agent_country.value_counts()

US    69172
MX       26
Name: ca_agent_country, dtype: int64

In [48]:
# MX here does not appear that useful
bexar_df[bexar_df.ca_agent_country=='MX']

,prop_id,prop_type_cd,prop_val_yr,geo_id,py_owner_id,py_owner_name,partial_owner,udi_group,py_addr_line1,py_addr_line2,...,Officer/Director State,deed_charter_diff,just_established_owner,foreign_based_owner,yoy_diff_2019,yoy_diff_2018,yoy_diff_2017,yoy_diff_2016,out_of_state_owner,entity_agent_binary
336540,586556,R,2019,15676-001-0110,456909,UNIFRID CORP,F,0,C/O RICARDO UNIKEL,3217 BURN BRAE DR,...,NaN,NaT,0,0,-1200.0,11300.0,200.0,2500.0,1,1
336541,586557,R,2019,15676-001-0120,456910,UNIFRID CORP,F,0,C/O RICARDO UNIKEL,3217 BURN BRAE DR,...,NaN,NaT,0,0,200.0,9900.0,200.0,2500.0,1,1
336542,586558,R,2019,15676-001-0130,456910,UNIFRID CORP,F,0,C/O RICARDO UNIKEL,3217 BURN BRAE DR,...,NaN,NaT,0,0,200.0,9900.0,200.0,2500.0,1,1
336543,586559,R,2019,15676-001-0140,456910,UNIFRID CORP,F,0,C/O RICARDO UNIKEL,3217 BURN BRAE DR,...,NaN,NaT,0,0,200.0,9900.0,200.0,2500.0,1,1
336544,586560,R,2019,15676-001-0150,456910,UNIFRID CORP,F,0,C/O RICARDO UNIKEL,3217 BURN BRAE DR,...,NaN,NaT,0,0,200.0,9900.0,200.0,2500.0,1,1
336545,586561,R,2019,15676-001-0160,456910,UNIFRID CORP,F,0,C/O RICARDO UNIKEL,3217 BURN BRAE DR,...,NaN,NaT,0,0,200.0,9900.0,200.0,2500.0,1,1
336546,586562,R,2019,15676-001-0170,456910,UNIFRID CORP,F,0,C/O RICARDO UNIKEL,3217 BURN BRAE DR,...,NaN,NaT,0,0,200.0,9900.0,200.0,2500.0,1,1
336547,586563,R,2019,15676-001-0180,456910,UNIFRID CORP,F,0,C/O RICARDO UNIKEL,3217 BURN BRAE DR,...,NaN,NaT,0,0,200.0,9900.0,200.0,2500.0,1,1
336548,586564,R,2019,15676-001-0190,456910,UNIFRID CORP,F,0,C/O RICARDO UNIKEL,3217 BURN BRAE DR,...,NaN,NaT,0,0,200.0,9900.0,200.0,2500.0,1,1
336549,586565,R,2019,15676-001-0200,456910,UNIFRID CORP,F,0,C/O RICARDO UNIKEL,3217 BURN BRAE DR,...,NaN,NaT,0,0,200.0,9900.0,200.0,-1700.0,1,1


In [49]:
# Dropping all chief appraiser data
bexar_df.drop(columns=['ca_agent_id', 'ca_agent_name', 'ca_agent_addr_line1',
                       'ca_agent_addr_line2', 'ca_agent_addr_line3', 'ca_agent_city',
                       'ca_agent_state', 'ca_agent_country', 'ca_agent_zip'],inplace=True)
print(bexar_df.shape)

(695510, 65)


##### DBA

In [52]:
len(bexar_df.dba.dropna())

72122

In [53]:
len(bexar_df['Taxpayer Name'].dropna())

59134

In [58]:
# Need to figure out how to make "owned by company" feature
bexar_df[['py_owner_id','py_owner_name','dba','Taxpayer Name']].head(20)

,py_owner_id,py_owner_name,dba,Taxpayer Name
0,83346,KEYSTONE SCHOOL,NaN,KEYSTONE SCHOOL
1,2851023,GONZALEZ MANUELA AGUILAR,NaN,NaN
2,2690996,ELMERS ICEHOUSE LLC,NaN,NaN
3,2690996,ELMERS ICEHOUSE LLC,NaN,NaN
4,85466,CITY OF SAN ANTONIO,NaN,NaN
5,70072,SAN ANTONIO RIVER AUTHORITY,NaN,NaN
6,83346,KEYSTONE SCHOOL,KEYSTONE SCHOOL,KEYSTONE SCHOOL
7,70026,TEXAS NEON ADVERTISING CO,SIGNS,NaN
8,70027,TEXAS NEON SIGN CO,TEXAS NEON SIGN CO,NaN
9,70030,CULLUM JAMES A JR,NaN,NaN


##### Appraiser Confidential

In [60]:
bexar_df['appr_confidential_flag'].value_counts()

F    693091
T      2419
Name: appr_confidential_flag, dtype: int64

In [65]:
# We should be able to assume that every property that has a confidential 
# flag 1 shouldn't have an owner listed
bexar_df[bexar_df['appr_confidential_flag']=='T'][['py_owner_id','py_owner_name','py_confidential_flag','appr_confidential_flag']]

,py_owner_id,py_owner_name,py_confidential_flag,appr_confidential_flag
311,3165269,NaN,1,T
911,2962299,NaN,1,T
1098,2601513,NaN,1,T
2052,2865736,NaN,1,T
2398,2501909,NaN,1,T
...,...,...,...,...
685567,147843,CONTINENTAL HOMES OF TEXAS LP,0,T
685585,3157172,NaN,1,T
689569,1958556,PERRY HOMES LLC,0,T
690679,197105,WEEKLEY HOMES LLC,0,T


In [62]:
bexar_df.py_owner_name.dropna().shape

(693086,)

In [63]:
bexar_df.py_owner_id.dropna().shape

(695510,)

In [66]:
# Numbers line up
bexar_df.py_confidential_flag.value_counts()

0    693086
1      2424
Name: py_confidential_flag, dtype: int64

In [67]:
# Convert appraiser confidential to 1/0
bexar_df['appr_confidential_flag'] = np.where(bexar_df['appr_confidential_flag'] == 'T',1,0)

##### Taxpayer State

In [71]:
bexar_df['Taxpayer State'].value_counts()

TX    51216
CA     2413
AZ      855
FL      634
CO      602
OH      540
AR      403
MI      335
MA      319
IL      299
NY      270
GA      263
VA      252
TN      226
MO      218
NJ      208
PA      176
NC      170
MN      157
LA      144
WA      142
VI      133
AK      129
OK      118
KY      114
IN       91
KS       89
NV       83
MD       77
RI       72
UT       67
AL       61
WI       60
CT       59
OR       50
MS       46
SC       45
NM       24
NE       24
ID       24
DC       24
HI       21
NH       17
WY       14
IA       13
SD        5
WV        3
MT        3
ME        2
DE        2
ND        2
PR        1
AE        1
VT        1
Name: Taxpayer State, dtype: int64

In [72]:
len(bexar_df['Taxpayer State'].dropna())
# Unclear what to do with this
# Already have if the owner is foreign-based and based outside of Texas

61317

##### Taxpayer Organizational Type

In [73]:
bexar_df['Taxpayer Organizational Type'].value_counts()

CL    24080
PL    10437
CT     8818
CI     7401
CF     3309
CN     2874
AR     2781
PF     1067
AP      186
PB      154
CP      141
CM      125
TR       42
TF       40
PV       39
PI       19
AF       19
TI       19
CU        4
AB        2
AC        1
Name: Taxpayer Organizational Type, dtype: int64

In [77]:
bexar_df[bexar_df['Taxpayer Organizational Type']=='HF'].head()
# No foreign holding companies

,prop_id,prop_type_cd,prop_val_yr,geo_id,py_owner_id,py_owner_name,partial_owner,udi_group,py_addr_line1,py_addr_line2,...,Officer/Director State,deed_charter_diff,just_established_owner,foreign_based_owner,yoy_diff_2019,yoy_diff_2018,yoy_diff_2017,yoy_diff_2016,out_of_state_owner,entity_agent_binary


In [81]:
bexar_df[bexar_df['Taxpayer Organizational Type']=='CF'][['py_owner_name','py_addr_country']][:10]

,py_owner_name,py_addr_country
289,HARCROS CHEMICALS INC,US
471,TINDALL CORPORATION,US
792,CVS PHARMACY INC,US
1170,ATM DEVELOPMENTS LTD,US
1625,SEASIDE HOSPITALITY CORPORATION,US
1638,OMNI LA MANSION CORPORATION,US
2958,PEREZ-SALINAS INC,US
8484,RF SERVICES INC,US
9279,PEREZ-SALINAS INC,US
9283,PEREZ-SALINAS INC,US


##### SOS Status Code

In [70]:
bexar_df['SOS Status Code'].value_counts()

A    52789
R     5083
F      177
W        6
C        6
Y        4
Name: SOS Status Code, dtype: int64

In [85]:
# Will use get dummies to make binary columns for each status
bexar_sos_status = pd.get_dummies(bexar_df['SOS Status Code'],prefix='sos_status_code_')

In [87]:
bexar_df = pd.concat([bexar_df,bexar_sos_status],axis=1)

##### Owner owns more than one property